# Reinforcement Learning 2023 : TRPO + PPO v2 + GAE
## Author : Jad El Karchi
This is an implementation

In [33]:
import numpy as np
import gym
import tensorflow as tf

print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))
# Function to compute the Hessian-vector product using Conjugate Gradient
def conjugate_gradient(Ax, b, max_iter=10, tol=1e-10):

    x = np.zeros_like(b)
    r = tf.identity(b)
    p = tf.identity(r)
    rho = np.dot(r, r)

    # to understand !
    for _ in range(max_iter):
        z = Ax(p)
        alpha = rho / (np.dot(p, z) + 1e-8)
        x += alpha * p
        r -= alpha * z
        rho_new = np.dot(r, r)
        beta = rho_new / (rho + 1e-8)
        p = r + beta * p
        rho = rho_new

        if rho < tol:
            break

    return x

# Function to run policy and collect trajectories
def run_policy(env, policy, num_trajectories):
    states, actions, rewards, log_probs = [], [], [], []

    for _ in range(num_trajectories):
        state = env.reset()[0]

        while True:
            # Sample action from the policy
            action_prob = policy(tf.expand_dims(state, axis=0))[0]
            action = np.random.choice(len(action_prob), p=action_prob.numpy())

            log_prob = tf.math.log(action_prob[action])
            next_state, reward, done, _ = env.step(action)[0]

            states.append(state)
            actions.append(action)
            rewards.append(reward)
            log_probs.append(log_prob)

            state = next_state

            if done:
                break
                
    return np.array(states), np.array(actions), np.array(rewards), np.array(log_probs)


# Function to compute advantage estimates
def calculate_advantages(rewards, gamma):
    advantages = np.zeros_like(rewards, dtype=np.float32)
    running_add = 0

    for t in reversed(range(len(rewards))):
        running_add += rewards[t]*gamma
        advantages[t] = running_add - np.mean(rewards)

    return advantages

def calculate_gae_advantages(rewards, values, gamma=0.99, lam=0.95):
    deltas = rewards[:-1] + gamma * values[1:] - values[:-1]
    advantages = np.zeros_like(rewards, dtype=np.float32)
    advantage = 0.0

    for t in reversed(range(len(deltas))):
        advantage = deltas[t] + gamma * lam * advantage
        advantages[t] = advantage

    return advantages

# Function to compute policy gradient
def compute_policy_gradient(policy, states, actions, advantages):
    with tf.GradientTape() as tape:
        logits = policy(states)
        action_masks = tf.one_hot(actions, depth=logits.shape[1])
        selected_logits = tf.reduce_sum(action_masks * logits, axis=1)
        loss = -tf.reduce_sum(selected_logits * advantages)
        
    return tape.gradient(loss, policy.trainable_variables)

# Main TRPO training loop
def trpo_training(env, policy, num_iterations=100, num_trajectories=10, max_timesteps=1000, gamma=0.99):
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

    for iteration in range(1, num_iterations + 1):
        # Run policy and collect trajectories
        states, actions, rewards, log_probs = run_policy(env, policy, num_trajectories)

        # Estimate advantage function
        advantages = calculate_advantages(rewards, gamma)

        # Compute policy gradient
        policy_gradient = compute_policy_gradient(policy, states, actions, advantages)

        # Use CG to compute F^-1 g
        def hessian_vector_product(vector):
            with tf.GradientTape() as tape2:
                with tf.GradientTape() as tape1:
                    logits = policy(states)
                    action_masks = tf.one_hot(actions, depth=logits.shape[1])
                    selected_logits = tf.reduce_sum(action_masks * logits, axis=1)
                    kl_divergence = tf.reduce_sum(tf.stop_gradient(tf.expand_dims(log_probs, axis=-1)) * (tf.expand_dims(log_probs, axis=-1) - tf.math.log(logits)), axis=1)

                gradient = tape1.gradient(selected_logits, policy.trainable_variables)
                gradient_vector_product = sum(tf.reduce_sum(g * v) for g, v in zip(gradient, vector))

                surrogate_loss = tf.reduce_mean(-selected_logits * advantages)
                kl_penalty = tf.reduce_mean(kl_divergence)

            hessian_vector_product_grad = tape2.gradient(gradient_vector_product, policy.trainable_variables)

            return hessian_vector_product_grad[3]

        hvp = conjugate_gradient(hessian_vector_product, policy_gradient[3])

        # Do line search on surrogate loss and KL constraint
        # (Not implemented here; typically a backtracking line search is used)

        # Update policy parameters
        optimizer.apply_gradients(zip(hvp, policy.trainable_variables))

        # Print or log some information
        if iteration % 100 == 0:
            total_reward = sum(rewards)
            print(f"Iteration {iteration}, Total Reward: {total_reward}")

# Define a simple policy network
class Policy(tf.keras.Model):
    def __init__(self, num_actions):
        super(Policy, self).__init__()
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(num_actions, activation='softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

# Create CartPole environment
swimmer_env = gym.make('CartPole-v1')

# Create policy network and optimizer
policy_model = Policy(num_actions=swimmer_env.action_space.n)

# Run TRPO training
trpo_training(swimmer_env, policy_model, num_iterations=10**3)

Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Iteration 100, Total Reward: -0.500572919845581
Iteration 200, Total Reward: 1.3723833411931992
Iteration 300, Total Reward: 0.3609631210565567
Iteration 400, Total Reward: -0.06682701408863068
Iteration 500, Total Reward: -0.339872345328331
Iteration 600, Total Reward: -0.7321433275938034
Iteration 700, Total Reward: 0.9249025136232376
Iteration 800, Total Reward: -0.19070617854595184
Iteration 900, Total Reward: -0.2717653959989548
Iteration 1000, Total Reward: -0.0396607369184494


In [34]:
def ppo_v1_training(env, policy, value_network, num_iterations=100, num_trajectories=10, max_timesteps=1000, epochs=10, clip_ratio=0.2, beta=0.01, gamma=0.99):
    optimizer_policy = tf.keras.optimizers.Adam(learning_rate=1e-3)
    optimizer_value = tf.keras.optimizers.Adam(learning_rate=1e-3)

    for iteration in range(1, num_iterations + 1):
        # Run policy and collect trajectories
        states, actions, rewards, log_probs = run_policy(env, policy, num_trajectories)

        # Train value network
        for epoch in range(epochs):
            values = value_network(states)
            advantages = calculate_advantages(rewards, gamma) #values)
            with tf.GradientTape() as tape_value:
                value_loss = tf.reduce_mean(tf.square(values - (rewards + advantages)))
            gradients_value = tape_value.gradient(value_loss, value_network.trainable_variables)
            optimizer_value.apply_gradients(zip(gradients_value, value_network.trainable_variables))

        # Train policy network using PPO objective
        for epoch in range(epochs):
            with tf.GradientTape() as tape_policy:
                new_log_probs = policy(states)
                ratio = tf.exp(new_log_probs - log_probs)
                clipped_ratio = tf.clip_by_value(ratio, 1 - clip_ratio, 1 + clip_ratio)
                surrogate = tf.minimum(ratio * advantages, clipped_ratio * advantages)
                policy_loss = -tf.reduce_mean(surrogate)
            gradients_policy = tape_policy.gradient(policy_loss, policy.trainable_variables)
            optimizer_policy.apply_gradients(zip(gradients_policy, policy.trainable_variables))

        # Dual descent update for beta
        entropy = -tf.reduce_sum(policy(states) * tf.math.log(policy(states)))
        with tf.GradientTape() as tape_beta:
            beta_loss = -beta * tf.reduce_mean(entropy)
        gradients_beta = tape_beta.gradient(beta_loss, policy.trainable_variables)
        optimizer_policy.apply_gradients(zip(gradients_beta, policy.trainable_variables))

        # Print or log some information
        if iteration % 10 == 0:
            total_reward = sum(rewards)
            print(f"Iteration {iteration}, Total Reward: {total_reward}")
